In [53]:
# Import the pathlib and csv library
from pathlib import Path
import csv

# Set the file paths
menu_filepath = Path("./Resources/menu_data.csv")
sales_filepath = Path("./Resources/sales_data.csv")

# Initialize a lists
menu = []
sales = []

# Open the csv file as an object
with open(menu_filepath, 'r') as csvfile:
    # Pass in the csv file to the csv.reader() function
    # (with ',' as the delmiter/separator) and return the csvreader object
    csvreader = csv.reader(csvfile, delimiter=",")
    # Read the header row first (skip this step if there is no header)
    header = next(csvreader)
    # Read each row of data after the header
    for row in csvreader:
        menu.append(row)
        
with open(sales_filepath, "r") as csvfile:
    # Pass in the csv file to the csv.reader() function
    # (with ',' as the delmiter/separator) and return the csvreader object
    csvreader = csv.reader(csvfile, delimiter=",") 
    # Read the header row first (skip this step if there is no header)
    header = next(csvreader)
    # Read each row of data after the header
    for row in csvreader:
        sales.append(row)
        
# @TODO: Initialize dict object to hold our key-value pairs of items and metrics
report = {}

# Initialize a row counter variable
row_count = 0

# @TODO: Loop over every row in the sales list object
for orders in sales:
    # Line_Item_ID,Date,Credit_Card_Number,Quantity,Menu_Item
    # @TODO: Initialize sales data variables
    quantity = int(orders[3])
    sales_item = orders[4]
    # If the item value not in the report, add it as a new entry with initialized metrics
    # Naming convention allows the keys to be ordered in logical fashion, count, revenue, cost, profit
    if not sales_item in report: 
        report[sales_item] = {}
        report[sales_item]['01-count'] = 0
        report[sales_item]['02-revenue'] = 0
        report[sales_item]['03-cogs'] = 0
        report[sales_item]['04-profit'] = 0 
    # @TODO: For every row in our sales data, loop over the menu records to determine a match
    for items in menu:
        # Item,Category,Description,Price,Cost
        # @TODO: Initialize menu data variables
        item = items[0]
        price = float(items[3])
        cost = float(items[4])
        # @TODO: Calculate profit of each item in the menu data
        profit = price - cost
        # @TODO: If the item value in our sales data is equal to the any of the items in the menu, then begin tracking metrics for that item
        if item == sales_item: 
            # @TODO: Print out matching menu data
            # @TODO: Cumulatively add up the metrics for each item key
            report[sales_item]["01-count"] += quantity
            report[sales_item]["02-revenue"] += price * quantity
            report[sales_item]["03-cogs"] += cost * quantity
            report[sales_item]["04-profit"] += profit * quantity
            break
        # @TODO: Else, the sales item does not equal any of the items in the menu data, therefore no match
        #elif not item in menu:
        #   print(f'{sales_item} does not equal {item}! NO MATCH!') 
    # @TODO: Increment the row counter by 1
    row_count += 1

# @TODO: Print total number of records in sales data
print(f'Total records processed: {row_count}')

# @TODO: Write out report to a text file (won't appear on the command line output)
# Set the header for report.csv
header = ['item', 'count', 'revenue', 'cogs', 'profit']

# Set the path for the report.csv
report_path = Path("report.csv")

# Open the output path as a file and pass into the 'csv.writer()' function
# Set the delimiter/separater as a ','
with open(report_path, "w") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=",")
    
    # Write the header as the first row
    csvwriter.writerow(header)
    
    # Loop over every key in analysis and write the associated key (name) and
    # nested key-value pairs (metrics)
    for row in report:
        csvwriter.writerow(
            [
                row,
                report[row]["01-count"],
                report[row]["02-revenue"],
                report[row]["03-cogs"],
                report[row]["04-profit"] 
            ])

Total records processed: 74124


In [54]:
report

{'spicy miso ramen': {'01-count': 9238,
  '02-revenue': 110856.0,
  '03-cogs': 46190.0,
  '04-profit': 64666.0},
 'tori paitan ramen': {'01-count': 9156,
  '02-revenue': 119028.0,
  '03-cogs': 54936.0,
  '04-profit': 64092.0},
 'truffle butter ramen': {'01-count': 8982,
  '02-revenue': 125748.0,
  '03-cogs': 62874.0,
  '04-profit': 62874.0},
 'tonkotsu ramen': {'01-count': 9288,
  '02-revenue': 120744.0,
  '03-cogs': 55728.0,
  '04-profit': 65016.0},
 'vegetarian spicy miso': {'01-count': 9216,
  '02-revenue': 110592.0,
  '03-cogs': 46080.0,
  '04-profit': 64512.0},
 'shio ramen': {'01-count': 9180,
  '02-revenue': 100980.0,
  '03-cogs': 45900.0,
  '04-profit': 55080.0},
 'miso crab ramen': {'01-count': 8890,
  '02-revenue': 106680.0,
  '03-cogs': 53340.0,
  '04-profit': 53340.0},
 'nagomi shoyu': {'01-count': 9132,
  '02-revenue': 100452.0,
  '03-cogs': 45660.0,
  '04-profit': 54792.0},
 'soft-shell miso crab ramen': {'01-count': 9130,
  '02-revenue': 127820.0,
  '03-cogs': 63910.0,
 

In [34]:
items

['black sesame creme brulee',
 'desserts',
 'house made black sesame infused creme brulee; complimentary hojicha',
 '6',
 '3']

In [46]:
report

{'spicy miso ramen': {'01-count': 9238,
  '02-revenue': 110856.0,
  '03-cogs': 46190.0,
  '04-profit': 64666.0},
 'tori paitan ramen': {'01-count': 9156,
  '02-revenue': 119028.0,
  '03-cogs': 54936.0,
  '04-profit': 64092.0},
 'truffle butter ramen': {'01-count': 8982,
  '02-revenue': 125748.0,
  '03-cogs': 62874.0,
  '04-profit': 62874.0},
 'tonkotsu ramen': {'01-count': 9288,
  '02-revenue': 120744.0,
  '03-cogs': 55728.0,
  '04-profit': 65016.0},
 'vegetarian spicy miso': {'01-count': 9216,
  '02-revenue': 110592.0,
  '03-cogs': 46080.0,
  '04-profit': 64512.0},
 'shio ramen': {'01-count': 9180,
  '02-revenue': 100980.0,
  '03-cogs': 45900.0,
  '04-profit': 55080.0},
 'miso crab ramen': {'01-count': 8890,
  '02-revenue': 106680.0,
  '03-cogs': 53340.0,
  '04-profit': 53340.0},
 'nagomi shoyu': {'01-count': 9132,
  '02-revenue': 100452.0,
  '03-cogs': 45660.0,
  '04-profit': 54792.0},
 'soft-shell miso crab ramen': {'01-count': 9130,
  '02-revenue': 127820.0,
  '03-cogs': 63910.0,
 